# Workshop 03

<b>Obiettivo</b>: Creare un modello 3D per la <i>chiesa centralizzata di Leonardo</i>.

![ws3_Leo.png](files/Immagini/ws3_Leo.png)

In [3]:
# Importo la libreria:
import sys
sys.path.insert(0, 'C:\Users\Camilla Bianca\Documents\GGPL\lar-cc\larlib')
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00699367768328 seconds


<hr>
- <b>Torre cilindrica</b>:

In [5]:
# l = larghezza; h = altezza; n = numero facce;
def cilindro(l,h,n):
    return TEXTURE('Texture/legno.jpg')(CYLINDER([l,h])(n))

VIEW(cilindro(8,15,10))

# Per ottenere una cupola faccio l'intersezione tra una sfera ed un cubo:
# d = diametro; fo = facce orizzontali; fv = facce verticali;
def cupola(d,fo,fv):
    sfera=JOIN(SPHERE(d)([fo,fv]))
    cubo=T([1,2])([-d,-d])(CUBOID([d*3,d*3,d*3]))
    cupola=INTERSECTION([cubo,sfera])
    # Abbellisco la cupola con due scalini nella parte bassa:
    baseBassa=CYLINDER([d*1.05,d*0.05])(fv)
    baseAlta=T(3)(d*0.1)(CYLINDER([d*1.1,d*0.1])(fv))
    cupolaCompleta=STRUCT([baseBassa,baseAlta,cupola])
    return TEXTURE('Texture/legno.jpg')(cupolaCompleta)

VIEW(cupola(8,15,8))

# Per ottenere una torre uniamo un cilindro ed una cupola:
# l = larghezza; h = altezza; n = numero facce;
def torre(l,h,n):
    i=int(n)
    cup=cupola(l,i,i)
    cil=cilindro(l,h/2,i)
    return TOP([cil,cup])

VIEW(torre(8,16,8))

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006EEE5D0> >

![ws3_cilindro.png](files/Immagini/ws3_cilindro.png)
![ws3_cupola.png](files/Immagini/ws3_cupola.png)
![ws3_torre.png](files/Immagini/ws3_torre.png)

<hr>
- <b>Puntale</b>:

In [6]:
def puntale(l,h):
    parteSuperiore=S(3)(1.5)(cupola(l,4,4))
    centro=cilindro(l,h/2,4)
    parteInferiore=cilindro(l*1.1,h*0.1,4)
    p=TOP([TOP([parteInferiore,centro]),parteSuperiore])
    return TEXTURE('Texture/legno.jpg')(p)

VIEW(puntale(1,2))

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E23F330> >

![ws3_puntale.png](files/Immagini/ws3_puntale.png)

<hr>
- <b>Base della struttura</b>:

In [7]:
# lBase = larghezza edificio;
# hBase = altezza edificio;
# lTorre = larghezza torre centrale;
# hTorre = altezza torre centrale;
def struttura(lBase,hBase,lTorre,hTorre,n):
    # t è la torre centrale, con il puntale in cima
    t=TOP([
        torre(lTorre,hTorre,n),
        puntale(lTorre/4,hTorre/4)
    ])
    # creo la struttura dell'edificio, con due "scalini": uno nel mezzo ed uno superiore
    piastra=CUBOID([lBase*1.05,lBase*1.05,hBase/10])
    base=TOP([TOP([CUBOID([lBase,lBase,(hBase/2)-(hBase/10)]),piastra]),CUBOID([lBase,lBase,(hBase/2)-(hBase/10)])])
    return TEXTURE('Texture/legno.jpg')(TOP([TOP([base,piastra]),t]))
    
VIEW(struttura(20,10,5,10,10))

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E23F090> >

![ws3_struttura.png](files/Immagini/ws3_struttura.png)

<hr>
- <b>Parte superiore della struttura</b>:

In [8]:
# La parte superiore della struttura è composta dalla struttura di base appena vista
# con l'aggiunta di quattro torri più piccole che si poggiano sulla piastra superiore:
def strutturaSuperiore(lBase,hBase,lTorre,hTorre,n):
    b=struttura(lBase,hBase,lTorre,hTorre,n)
    # t è una torre da ripetere 4 volte, più piccola di quella centrale:
    t=torre(lTorre/2,hTorre/2,n)
    torri=STRUCT([
        b,
        T([1,2,3])([lBase*0.15,lBase*0.15,hBase])(t),
        T([1,2,3])([lBase*0.85,lBase*0.15,hBase])(t),
        T([1,2,3])([lBase*0.15,lBase*0.85,hBase])(t),
        T([1,2,3])([lBase*0.85,lBase*0.85,hBase])(t)
    ])
    return torri

VIEW(strutturaSuperiore(20,10,5,10,8))

Child 3
Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006EE7CF0> >

![ws3_strutturaS.png](files/Immagini/ws3_strutturaS.png)

<hr>
- <b>Parte laterale della struttura</b>:

In [9]:
# La struttura laterale è praticamente una struttura base, ma più ristretta
# e con l'aggiunta di quattro mezze torri inserite nelle pareti laterali:
def strutturaLaterale(lBase,hBase,lTorre,hTorre,n):
    b=struttura(lBase,hBase,lTorre,hTorre,n)
    # t è una torre da ripetere 4 volte, più piccola di quella centrale:
    t=torre(lTorre*0.8,hTorre*2,n)
    s=STRUCT([
        b,
        T(1)(lBase/2)(t),
        T(2)(lBase/2)(t),
        T([1,2])([lBase,lBase/2])(t),
        T([1,2])([lBase/2,lBase])(t)
    ])
    return s

VIEW(strutturaLaterale(5,8,2,5,8))

Child 3
Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006EE7D20> >

![ws3_strutturaL_1.png](files/Immagini/ws3_strutturaL_1.png)

![ws3_strutturaL_2.png](files/Immagini/ws3_strutturaL_2.png)

<hr>
- <b>Struttura finale</b>:

In [10]:
# base(l) è una funzione che crea la base del modellino della chiesa:
def base(l):
    return TEXTURE('Texture/legno.jpg')(STRUCT([
        T([1,2])([-l/4,-l/4])(CUBOID([l+l/2,l+l/2,0.5])),
    ]))

# Funzione che crea la struttura intera:
def chiesaDaVinci(lBase,hBase,lTorre,hTorre,n):
    s=strutturaSuperiore(lBase,hBase,lTorre,hTorre,n)
    # misure delle parti laterali:
    htl=int(hTorre/1.5)
    lbl=lBase/3
    hbl=hBase/2
    ltl=lTorre/3
    l=strutturaLaterale(lbl,hbl,ltl,htl,n)
    b=base(lBase)
    return STRUCT([
        s,
        b,
        T([1,2])([lBase/2-lbl/2,-lbl/2])(l),
        T([1,2])(([-lbl/2,lBase/2-lbl/2]))(l),
        T([1,2])([lBase-lbl/2,lBase/2-lbl/2])(l),
        T([1,2])([lBase/2-lbl/2,lBase-lbl/2])(l),
    ])
   
VIEW(chiesaDaVinci(40,20,10,20,10))

Child 3
Child 3
Child 3
Child 3
Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E23F450> >

![ws3_DaVinci_1.png](files/Immagini/ws3_DaVinci_1.png)

![ws3_DaVinci_2.png](files/Immagini/ws3_DaVinci_2.png)